# Exercise 6

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [3]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
620,6.5,0.46,0.41,16.8,0.084,59.0,222.0,0.99930,3.18,0.58,9.0,5,white
1356,7.3,0.22,0.41,15.4,0.050,55.0,191.0,1.00000,3.32,0.59,8.9,6,white
5565,11.3,0.34,0.45,2.0,0.082,6.0,15.0,0.99880,2.94,0.66,9.2,6,red
3748,6.2,0.24,0.25,12.5,0.055,47.0,134.0,0.99758,3.30,0.51,9.0,5,white
4591,7.0,0.33,0.78,9.9,0.042,21.0,251.0,0.99435,3.01,0.55,11.0,6,white


# Exercise 6.1

Show the frecuency table of the quality by type of wine

In [4]:
freq = pd.value_counts(data.type).to_frame().reset_index()
freq.columns = ['Type','Count']
freq

,Type,Count
0,white,4898
1,red,1599


# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


In [37]:
#Standarize the features (not the quality)

data['quality_bool'] = data['quality'] > 6
#data_r['quality_bool'] = data_r['quality'] > 6
#data_w['quality_bool'] = data_w['quality'] > 6
datared = data[data['type'] == 'red']
datawhite = data[data['type'] == 'white']

y = data['quality_bool']
X = data.drop(['quality_bool', 'type', 'quality'],1)

from sklearn.preprocessing import StandardScaler
ss1 = StandardScaler(with_mean=True, with_std=True)
ss1.fit(X.astype(np.float))
X = ss1.transform(X.astype(np.float))
#ss.mean_, ss.scale_

y_red = datared['quality_bool']
X_red = datared.drop(['quality_bool', 'type', 'quality'],1)

from sklearn.preprocessing import StandardScaler
ss2 = StandardScaler(with_mean=True, with_std=True)
ss2.fit(X_red.astype(np.float))
X_red = ss2.transform(X_red.astype(np.float))
#ss.mean_, ss.scale_

y_white = datawhite['quality_bool']
X_white = datawhite.drop(['quality_bool', 'type', 'quality'],1)

from sklearn.preprocessing import StandardScaler
ss3 = StandardScaler(with_mean=True, with_std=True)
ss3.fit(X_white.astype(np.float))
X_white = ss3.transform(X_white.astype(np.float))
#ss.mean_, ss.scale_

In [38]:
#Create a binary target for each type of wine
data['type_bin'] = data['type'].replace({'white':1, 'red':0})
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,quality_bool,type_bin
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white,False,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white,False,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white,False,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white,False,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white,False,1


In [41]:
#Create two Linear SVM's for the white and red wines, repectively
from sklearn.svm import SVC

svm_white = SVC(kernel= 'linear')
svm_white.fit(X_white,y_white)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [42]:
svm_red = SVC(kernel= 'linear')
svm_red.fit(X_red,y_red)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


In [43]:
# SVM for white wines

svm_white_poly = SVC(kernel= 'poly')
svm_white_poly.fit(X_white,y_white)
score_white_poly = svm_white_poly.score(X_white,y_white)

svm_white_rbf = SVC(kernel= 'rbf')
svm_white_rbf.fit(X_white,y_white)
score_white_rbf = svm_white_rbf.score(X_white,y_white)

svm_white_sigmoid = SVC(kernel= 'sigmoid')
svm_white_sigmoid.fit(X_white,y_white)
score_white_sigmoid = svm_white_sigmoid.score(X_white,y_white)

# SVM for red wines

svm_red_poly = SVC(kernel= 'poly')
svm_red_poly.fit(X_red,y_red)
score_red_poly = svm_red_poly.score(X_red,y_red)

svm_red_rbf = SVC(kernel= 'rbf')
svm_red_rbf.fit(X_red,y_red)
score_red_rbf = svm_red_rbf.score(X_red,y_red)

svm_red_sigmoid = SVC(kernel= 'sigmoid')
svm_red_sigmoid.fit(X_red,y_red)
score_red_sigmoid = svm_red_sigmoid.score(X_red,y_red)

print('The scores for the white wine SVM using the different kernels are ','poly:', score_white_poly,'rbf:', score_white_rbf,'sigmoid:', score_white_sigmoid   )
print('The scores for the red wine SVM using the different kernels are ','poly:', score_red_poly,'rbf:', score_red_rbf,'sigmoid:', score_red_sigmoid   )




/Users/davidfajardo/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/davidfajardo/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/davidfajardo/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


The scores for the white wine SVM using the different kernels are  poly: 0.8164556962025317 rbf: 0.8397305022458146 sigmoid: 0.7354022049816251
The scores for the red wine SVM using the different kernels are  poly: 0.9061913696060038 rbf: 0.8993120700437773 sigmoid: 0.8186366479049406


/Users/davidfajardo/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/davidfajardo/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/davidfajardo/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

In [44]:
white_list = []
red_list = []

for c in [0.1, 1, 10, 100, 1000]:
    for g in [0.01, 0.001, 0.0001]:
        svm_white_rbf = SVC(kernel= 'rbf',C=c, gamma=g)
        svm_white_rbf.fit(X_white,y_white)
        white_list.append([c,g,svm_white_rbf.score(X_white,y_white)]) 
        svm_red_poly = SVC(kernel= 'poly',C=c, gamma=g)
        svm_red_poly.fit(X_red,y_red)
        red_list.append([c,g, svm_red_poly.score(X_red,y_red)])

white_list.sort(key=lambda x: x[2]) 
red_list.sort(key=lambda x: x[2]) 
print(white_list)
print(red_list)
print("La mejor combinacion para ambos tipos es c=1000 y gamma=0.01")

[[0.1, 0.01, 0.7835851367905268], [0.1, 0.001, 0.7835851367905268], [0.1, 0.0001, 0.7835851367905268], [1, 0.001, 0.7835851367905268], [1, 0.0001, 0.7835851367905268], [10, 0.001, 0.7835851367905268], [10, 0.0001, 0.7835851367905268], [100, 0.0001, 0.7835851367905268], [1000, 0.0001, 0.7835851367905268], [1, 0.01, 0.8007349938750511], [100, 0.001, 0.8064516129032258], [1000, 0.001, 0.8182931808901592], [10, 0.01, 0.819314005716619], [100, 0.01, 0.8344222131482237], [1000, 0.01, 0.8468762760310331]]
[[0.1, 0.01, 0.8642901813633521], [0.1, 0.001, 0.8642901813633521], [0.1, 0.0001, 0.8642901813633521], [1, 0.01, 0.8642901813633521], [1, 0.001, 0.8642901813633521], [1, 0.0001, 0.8642901813633521], [10, 0.001, 0.8642901813633521], [10, 0.0001, 0.8642901813633521], [100, 0.001, 0.8642901813633521], [100, 0.0001, 0.8642901813633521], [1000, 0.001, 0.8642901813633521], [1000, 0.0001, 0.8642901813633521], [10, 0.01, 0.8655409631019387], [100, 0.01, 0.8799249530956847], [1000, 0.01, 0.9105691056

# Exercise 6.5

Compare the results with other methods

In [47]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

reg_white = linear_model.LogisticRegression()
reg_white.fit(X_white,y_white)

reg_red = linear_model.LogisticRegression()
reg_red.fit(X_red,y_red)

predictions_white = reg_white.predict(X_white)
predictions_red = reg_white.predict(X_red)
print("Logistic Reg. for white, accuracy:", accuracy_score(y_white, predictions_white))
print("Logistic Reg. for red, accuracy:",accuracy_score(y_red, predictions_red))

print('''SVM es mejor para los dos tipos de vino:  
white_SVM=0.85 vs white_LOGIT=0.8
red_SVM=0.91 vs white_LOGIT=0.7986''')

Logistic Reg. for white, accuracy: 0.8017558187015108
Logistic Reg. for red, accuracy: 0.7986241400875547
SVM es mejor para los dos tipos de vino:  
white_SVM=0.85 vs white_LOGIT=0.8
red_SVM=0.91 vs white_LOGIT=0.7986


/Users/davidfajardo/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/davidfajardo/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous)

* Analyze the coefficients

* Evaluate the RMSE

In [49]:
from sklearn.model_selection import train_test_split

lr_X = X
lr_y = data['quality']

#Standarize y
y_mean, y_std = lr_y.mean(), lr_y.std()
lr_y = (lr_y - y_mean)/ y_std

X_train, X_test, y_train, y_test = train_test_split(lr_X, lr_y, test_size=0.2, random_state=42)

lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

lr.coef_

array([ 0.11887781, -0.27826199, -0.00759825,  0.33225392, -0.02617215,
        0.10735663, -0.10596155, -0.33067241,  0.08229493,  0.1250517 ,
        0.31047882, -0.16917504])

In [50]:
#RMSE
from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test,y_pred )))

RMSE: 0.7867684897961288


# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [53]:
# Ridge
from sklearn.linear_model import Ridge
ridgereg01 = Ridge(alpha=0.1)
ridgereg01.fit(X_train, y_train)
y_pred01 = ridgereg01.predict(X_test)
print('RMSE with alpha 0.1:', np.sqrt(metrics.mean_squared_error(y_test,y_pred01 )))

RMSE with alpha 0.1: 0.7867697144798156


In [54]:
ridgereg1 = Ridge(alpha=1)
ridgereg1.fit(X_train, y_train)
y_pred1 = ridgereg1.predict(X_test)
print('RMSE with alpha 1:', np.sqrt(metrics.mean_squared_error(y_test,y_pred1 )))

RMSE with alpha 1: 0.7867807516615621


# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [55]:
# Lasso
from sklearn.linear_model import Lasso
from sklearn.metrics import classification_report

lassoreg001 = Lasso(alpha=0.01)
lassoreg001.fit(X_train, y_train)
y_pred001 = lassoreg001.predict(X_test)
print('RMSE with alpha 0.01:', np.sqrt(metrics.mean_squared_error(y_test,y_pred001 )))

RMSE with alpha 0.01: 0.792323218765886


In [56]:
lassoreg01 = Lasso(alpha=0.1)
lassoreg01.fit(X_train, y_train)
y_pred01 = lassoreg01.predict(X_test)
print('RMSE with alpha 0.1:', np.sqrt(metrics.mean_squared_error(y_test,y_pred01 )))

RMSE with alpha 0.1: 0.8214188635497301


In [57]:
lassoreg1 = Lasso(alpha=1)
lassoreg1.fit(X_train, y_train)
y_pred1 = lassoreg1.predict(X_test)
print('RMSE with alpha 1:', np.sqrt(metrics.mean_squared_error(y_test,y_pred1 )))


RMSE with alpha 1: 0.9681743922374527


# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

In [58]:
from sklearn.metrics import f1_score
#Create a binary target
y = data['quality_bool']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = linear_model.LogisticRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

lr.coef_

print('The accuracy score is',accuracy_score(y_test, y_pred),', which means this model has a good accuracy.')
print('The f1 score is',f1_score(y_test, y_pred), 'which means the model wouldn\'t be the best, the closer to 1 the better.')

The accuracy score is 0.8184615384615385 , which means this model has a good accuracy.
The f1 score is 0.34806629834254144 which means the model wouldn't be the best, the closer to 1 the better.


/Users/davidfajardo/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score

In [59]:
results = []

for c in [0.01, 0.1, 1]:
    for p in ['l1', 'l2']:
        logreg = linear_model.LogisticRegression(C=c, penalty = p, solver='liblinear',multi_class='auto')
        logreg.fit(X_train,y_train)
        y_pred = logreg.predict(X_test)   
        results.append([c,p,f1_score(y_test, y_pred)])

print(results)
print("La mejor combinacion para ambos tipos es c=1 y penalty=l2")

[[0.01, 'l1', 0.24281150159744405], [0.01, 'l2', 0.3285302593659943], [0.1, 'l1', 0.32386363636363635], [0.1, 'l2', 0.3388888888888889], [1, 'l1', 0.34349030470914127], [1, 'l2', 0.34806629834254144]]
La mejor combinacion para ambos tipos es c=1000 y gamma=0.01
